In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


In [15]:
from fuel.datasets.cifar10 import CIFAR10
from fuel.transformers import ScaleAndShift, Cast, Flatten, Mapping
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme, ShuffledScheme

train_batch_size = 100
validation_batch_size = 250


CIFAR10.default_transformers = (
    (ScaleAndShift, [2.0 / 255.0, -1], {'which_sources': 'features'}),
    (Cast, [np.float32], {'which_sources': 'features'}), 
    #(Flatten, [], {'which_sources': 'features'}),
    #(Flatten, [], {'which_sources': 'targets'}),
    #(Mapping, [lambda batch: (b.T for b in batch)], {}) 
    )

cifar_train = CIFAR10(("train",), subset=slice(None,10000))
cifar_train_stream = DataStream.default_stream(
    cifar_train,
    iteration_scheme=ShuffledScheme(cifar_train.num_examples, train_batch_size))

cifar_validation = CIFAR10(("train",), subset=slice(99000, None))
cifar_validation_stream = DataStream.default_stream(
    cifar_validation, iteration_scheme=SequentialScheme(cifar_validation.num_examples, validation_batch_size))

cifar_test = CIFAR10(("test",))
cifar_test_stream = DataStream.default_stream(
    cifar_test, iteration_scheme=SequentialScheme(cifar_test.num_examples, validation_batch_size))

print "The streams return batches containing %s" % (cifar_train_stream.sources,)

print "Each trainin batch consits of a tuple containing:"
for element in next(cifar_train_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)
    
print "Validation/test batches consits of tuples containing:"
for element in next(cifar_test_stream.get_epoch_iterator()):
    print " - an array of size %s containing %s" % (element.shape, element.dtype)

cifar_labels = ["airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

The streams return batches containing (u'features', u'targets')
Each trainin batch consits of a tuple containing:
 - an array of size (100, 3, 32, 32) containing float32
 - an array of size (100, 1) containing uint8
Validation/test batches consits of tuples containing:
 - an array of size (250, 3, 32, 32) containing float32
 - an array of size (250, 1) containing uint8


In [33]:
# create Theano variables for input and target minibatch
input_var = T.tensor4('X')
target_var = T.ivector('y')

# create a small convolutional neural network
from lasagne.nonlinearities import leaky_rectify, softmax
network = lasagne.layers.InputLayer((None, 3, 32, 32), input_var)
network = lasagne.layers.Conv2DLayer(network, 64, (3, 3),
                                     nonlinearity=leaky_rectify, pad='same',
                                     W=lasagne.init.HeNormal(gain=sqrt(2/(1.0001))))
network = lasagne.layers.Conv2DLayer(network, 64, (3, 3),
                                     nonlinearity=leaky_rectify, pad='same',
                                     W=lasagne.init.HeNormal(gain=sqrt(2/(1.0001))))
network = lasagne.layers.Pool2DLayer(network, (2, 2), stride=2, mode='max')
network = lasagne.layers.Conv2DLayer(network, 128, (3, 3),
                                     nonlinearity=leaky_rectify, pad='same',
                                     W=lasagne.init.HeNormal(gain=sqrt(2/(1.0001))))
network = lasagne.layers.Conv2DLayer(network, 128, (3, 3),
                                     nonlinearity=leaky_rectify, pad='same',
                                     W=lasagne.init.HeNormal(gain=sqrt(2/(1.0001))))
network = lasagne.layers.Pool2DLayer(network, (2, 2), stride=2, mode='max')
network = lasagne.layers.Conv2DLayer(network, 256, (3, 3),
                                     nonlinearity=leaky_rectify, pad='same',
                                     W=lasagne.init.HeNormal(gain=sqrt(2/(1.0001))))
network = lasagne.layers.Conv2DLayer(network, 256, (3, 3),
                                     nonlinearity=leaky_rectify, pad='same',
                                     W=lasagne.init.HeNormal(gain=sqrt(2/(1.0001))))
network = lasagne.layers.Pool2DLayer(network, (2, 2), stride=2, mode='max')
# network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5), nonlinearity=None,
#                                     num_units=2048, W=lasagne.init.Orthogonal())
# network = lasagne.layers.FeaturePoolLayer(network, 16)
# network = lasagne.layers.DenseLayer(network, nonlinearity=None,
#                                     num_units=2048, W=lasagne.init.Orthogonal())
# network = lasagne.layers.FeaturePoolLayer(network, 16)
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                   1024, nonlinearity=leaky_rectify,
#                                    W=lasagne.init.GlorotNormal(gain=sqrt(2/(1.0001))))
                                   W=lasagne.init.Orthogonal())
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                   1024, nonlinearity=leaky_rectify,
#                                    W=lasagne.init.GlorotNormal(gain=sqrt(2/(1.0001))))
                                   W=lasagne.init.Orthogonal())
network = lasagne.layers.DenseLayer(lasagne.layers.dropout(network, 0.5),
                                    10, nonlinearity=softmax)

# create loss function
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean() + 1e-4 * lasagne.regularization.regularize_network_params(
        network, lasagne.regularization.l2)

a0 = np.float32(0.02)
tau = np.float32(cifar_train.num_examples)
learning_rate = theano.shared(np.array(a0, dtype=config.floatX))
t = T.scalar()
anneal_learning_rate = theano.function([t], None, updates=[
        (learning_rate, a0 * (tau / T.max([t, tau])))])

# create parameter update expressions
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=learning_rate,
                                            momentum=0.9)
#updates = lasagne.updates.rmsprop(loss, params, learning_rate=0.1)
# compile training function that updates parameters and returns training loss
train_fn = theano.function([input_var, target_var], loss, updates=updates)


test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var), dtype=theano.config.floatX)

val_fn = theano.function([input_var, target_var], [test_acc])


In [36]:
best_val_acc = 0
best_params = lasagne.layers.get_all_param_values(network)
best_params_epoch = 0

last_expansion_epoch = 0
expansion_coef = 1.5
    
num_epochs = 10

learning_rate.set_value(a0)

print("Starting training...")
epoch = 0
t = 0
while epoch < num_epochs:
    epoch += 1
    train_err = 0
    train_batches = 0
    start_time = time.time()
    
    print("Learning rate: {:.6f}".format(float(learning_rate.get_value())))
    
    for input_batch, target_batch in cifar_train_stream.get_epoch_iterator():
        train_err += train_fn(input_batch, target_batch.ravel())
        train_batches += 1

    t += cifar_train.num_examples
    anneal_learning_rate(t)
    
    
    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for input_batch, target_batch in cifar_validation_stream.get_epoch_iterator():
        acc = val_fn(input_batch, target_batch.ravel())[0]
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

    
    # patience expansion
    if val_acc > best_val_acc:
        best_params = lasagne.layers.get_all_param_values(network)
        best_params_epoch = epoch
        best_val_acc = val_acc
        num_epochs = int(max(epoch * expansion_coef, epoch + 10))
        best_val_err = val_err
        last_expansion_epoch = epoch
        print "After epoch %d: increased planned number of epochs to %d" % (epoch, num_epochs)

    if epoch - last_expansion_epoch >= 5:
        t *= 2
        anneal_learning_rate(t)
        last_expansion_epoch = epoch
        print "Learning is stalling, reducing rate by half."


Starting training...
Learning rate: 0.020000
Epoch 1 of 10 took 22.522s
  training loss:		1.685606
  validation accuracy:		57.80 %
After epoch 1: increased planned number of epochs to 11
Learning rate: 0.020000
Epoch 2 of 11 took 22.490s
  training loss:		1.528172
  validation accuracy:		58.80 %
After epoch 2: increased planned number of epochs to 12
Learning rate: 0.010000
Epoch 3 of 12 took 22.492s
  training loss:		1.339072
  validation accuracy:		62.30 %
After epoch 3: increased planned number of epochs to 13
Learning rate: 0.006667
Epoch 4 of 13 took 22.492s
  training loss:		1.203942
  validation accuracy:		63.20 %
After epoch 4: increased planned number of epochs to 14
Learning rate: 0.005000
Epoch 5 of 14 took 22.491s
  training loss:		1.114908
  validation accuracy:		64.50 %
After epoch 5: increased planned number of epochs to 15
Learning rate: 0.004000
Epoch 6 of 15 took 22.491s
  training loss:		1.041265
  validation accuracy:		66.00 %
After epoch 6: increased planned number

In [6]:
# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for input_batch, target_batch in cifar_test_stream.get_epoch_iterator():
    err, acc = val_fn(input_batch, target_batch.ravel())
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))


lasagne.layers.set_all_param_values(network, best_params)

test_err = 0
test_acc = 0
test_batches = 0
for input_batch, target_batch in cifar_test_stream.get_epoch_iterator():
    err, acc = val_fn(input_batch, target_batch.ravel())
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results (using best_params from epoch {}):".format(best_params_epoch))
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))


Final results:
  test loss:			1.274424
  test accuracy:		85.89 %
Final results (using best_params from epoch 20):
  test loss:			1.263442
  test accuracy:		85.89 %
